Analysis of LiLa field test data
-----------
Author: Albert Ulmer  
Date: May 2022  

In [ ]:
# autoreload packages
%load_ext autoreload
%autoreload 2

# libraries
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, Any
import datetime as dt

import myfunctions as mf


In [ ]:
# parameters
phases = {}
phases["phase1"] = pd.Timestamp("2022-03-07 00:00:00.000000")
phases["phase2"] = pd.Timestamp("2022-03-14 00:00:00.000000")
phases["phase3"] = pd.Timestamp("2022-03-21 00:00:00.000000")
phases["phase4"] = pd.Timestamp("2022-03-28 00:00:00.000000")
phases["end"] = pd.Timestamp("2022-04-04 00:00:00.000000")

# constants
freq = '15T'

In [ ]:
# connect to SQLite database
try:
    conn = sqlite3.connect('../../data/lila/purple.sqlite3')
    print('Connected to database...')
except:
    print('Database error!')
    exit()


In [ ]:
### Set global flag whether to save plots to files or not
writefiles = 1

if writefiles:
    print("Writing output files!")
else:
    print("Leaving files alone!")


In [ ]:
### Set global flag whether to print debug messages while running code
showdebug = 1

if showdebug:
    print("Showing debug messages!")
else:
    print("No debug messages will be shown!")

## Charging Comfort

In [ ]:
# execute SQL query
querycc = open("sqls/cars_with_wallboxes_15min_v2.sql").read()
dfcc = pd.read_sql_query(querycc, conn)

# fix datatypes
dfcc["timestamp"] =  pd.to_datetime(dfcc["timestamp"]) 

# filter to daterange of field test
dfcc = dfcc[(dfcc["timestamp"] >= phases["phase1"]) & (dfcc["timestamp"] < phases["end"])]

# set index
dfcc.set_index(['vehicle', 'timestamp'], inplace=True, drop=True)
dfcc.head()


In [ ]:
# load EV data and add features
dfcc2 = mf.load_ev_data(dfcc, phases, freq)
dfcc2.head()


In [ ]:
# find first departure per day
dfcc3 = mf.filter_ev_data_first_departure_per_day(dfcc2)
dfcc3


In [ ]:
# reset index for easier analysis and plotting
dfcc4 = dfcc3.reset_index(inplace=False)
dfcc4


#### Satisfaction per car

In [ ]:
# prepare data for plotting
dfcc4_pivot = dfcc4[dfcc4.stateOfCharge>30].pivot_table(index="phase", values="stateOfCharge", columns="vehicle").round(2)
models = ["Direct", "Rule-based", "Predictive", "Stochastic"]
models.reverse()
dfcc4_pivot.sort_index(ascending=False, inplace=True)
dfcc4_pivot.index = models
dfcc4_pivot

In [ ]:
plt.style.use("ggplot")
f, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(data=dfcc4_pivot, annot=True, fmt=".0f", ax=ax, cmap="Greens", cbar_kws={"label": "Satisfaction [%]"})
ax.set_xlabel("Vehicle")
ax.set_ylabel("Charging Strategy")
#ax.set_zlabel("Satisfaction [%]")
if writefiles:
    plt.savefig("output/chargingcomfort_cars.png",
                bbox_inches='tight', dpi=300)
    plt.close()


#### Satisfaction per strategy

In [ ]:
dfcc4_pivot_strategies = dfcc4[dfcc4.stateOfCharge>30].pivot_table(values="stateOfCharge", index="phase", aggfunc=["max", "min", "std", "median"]).round(2)
models = ["Direct", "Rule-based", "Predictive", "Stochastic"]
dfcc4_pivot_strategies.index = models
dfcc4_pivot_strategies.columns.set_levels(["Maximum", "Minimum", "Std. Deviation", "Median"], level=0, inplace=True)
dfcc4_pivot_strategies.columns.set_levels(["Satisfaction [%]"], level=1, inplace=True)
dfcc4_pivot_strategies

In [ ]:
if writefiles:
    dfcc4_pivot_strategies.to_latex(
        buf="output/chargingcomfort_strategies.tex", bold_rows=True)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

models = ["Direct", "Rule-based", "Predictive", "Stochastic"]
plt.style.use("ggplot")

f, ax = plt.subplots()
sns.violinplot(data=dfcc4[dfcc4.stateOfCharge>30], x="phase", y="stateOfCharge", #palette="light:g",
               alpha=1, bw=.2, cut=1, linewidth=2)
ax.set_xlabel("Charging Strategy")
ax.set_ylabel("Satisfaction [%]")
ax.set_xticklabels(models)
#ax.set(ylim=(0, 100))

if writefiles:
    plt.savefig("output/chargingcomfort_strategies.png",
                bbox_inches='tight', dpi=300)
    #plt.close()


### Departure Time per Vehicle

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")

f, ax = plt.subplots()
sns.violinplot(data=dfcc4, x="vehicle", y="hour",
               alpha=1, bw=.2, cut=1, linewidth=2)
ax.set_xlabel("Vehicle")
ax.set_ylabel("Hour of departure")
ax.set(ylim=(0, 24))

if writefiles:
    plt.savefig("output/departuretimes_cars.png",
                bbox_inches='tight', dpi=300)
    #plt.close()


### Driving vs. Charging

In [ ]:
#generate heat feature for plotting: heat = 1 ... charging, -1 ... driving
dfcc5 = dfcc2.reset_index()
dfcc5["heat"] = dfcc5.charging - dfcc5.driving
dfcc5.head()

In [ ]:
def car_heatmap(data, vehicle, phase, ax):
    my_ev = data[(data.vehicle == vehicle) & (data.phase == phase)]
    my_ev = my_ev.set_index("timestamp")
    my_ev = my_ev.resample("H").agg(
             {"heat": "mean"
              }
         ).pad()
    my_ev = my_ev.reset_index()

    #hour = my_ev["timestamp"].dt.hour
    date = my_ev["timestamp"].dt.date
    temp = my_ev["heat"]
    temp = temp.values.reshape(24, len(date.unique()), order="F")
    
    xgrid = np.arange(len(date.unique())+ 1) + 1
    ygrid = np.arange(25)
    
    ax.grid(False)
    ax.pcolormesh(xgrid, ygrid, temp, cmap="RdYlGn", vmin=MIN_TEMP, vmax=MAX_TEMP)
    # Invert the vertical axis
    ax.set_ylim(24, 0)
    # Set tick positions for both axes
    #ax.yaxis.set_ticks([i for i in range(24)])
    ax.yaxis.set_ticks([0,6,12,18])
    ax.xaxis.set_ticks([i+1 for i in range(7)])
    # Remove ticks by setting their length to 0
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    
    # Remove all spines
    ax.set_frame_on(False)

    return ax


In [ ]:
MIN_TEMP = dfcc5["heat"].min()
MAX_TEMP = dfcc5["heat"].max()

my_vehicles = dfcc5.vehicle.unique()
my_phases = [1,2,3,4]
models = ["Direct", "Rule-based", "Predictive", "Stochastic"]

fig, axes = plt.subplots(len(my_phases), len(my_vehicles), figsize=(7, 7), sharey=True)

for i, p in enumerate(my_phases):
    for j, v in enumerate(my_vehicles):
        myhm = car_heatmap(dfcc5, v, p, axes[i, j])
        if i == 0:
            myhm.set_title(v)
        if i == len(my_phases) - 1:
            myhm.set_xlabel("Day of Week", fontsize=8)
        if j == 0:
            myhm.set_ylabel("Hour of Day", fontsize=8)
        if j == len(my_vehicles) - 1:
            myhm.yaxis.set_label_position("right")
            #myhm.yaxis.tick_right()
            myhm.set_ylabel(models[i])

if writefiles:
    plt.savefig("output/charging_driving_cars.png",
                bbox_inches='tight', dpi=300)
    #plt.close()



## Peak Shaving

In [ ]:
# execute SQL query
querypeak = open("sqls/opt_runs_15min.sql").read()
dfpeak = pd.read_sql_query(querypeak, conn)

# fix datatypes
dfpeak["runningdate"] =  pd.to_datetime(dfpeak["runningdate"]) 

# filter to daterange of field test
dfpeak = dfpeak[(dfpeak["runningdate"] >= phases["phase1"]) & (dfpeak["runningdate"] < phases["end"])]
dfpeak = dfpeak.set_index("runningdate")
dfpeak.head()

In [ ]:
dfpeak2 = dfpeak["GridDraw"].resample('W').agg(["max", "mean", "std", mf.papr]).pad().round(2)
models = ["Direct", "Rule-based", "Predictive", "Stochastic"]
metrics = ["Maximum [kW]", "Average [kW]", "Std. Deviation [kW]", "PAPR"]
dfpeak2.index = models
dfpeak2.columns = metrics
dfpeak2 = dfpeak2 #.T #.reset_index()
dfpeak2

In [ ]:
if writefiles:
    dfpeak2.to_latex(
        buf="output/peakshaving_strategies.tex", bold_rows=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")

f, ax = plt.subplots()
sns.violinplot(data=dfpeak, x="phase", y="GridDraw",
               alpha=1, bw=.2, cut=1, linewidth=2)
ax.set_xlabel("Charging Strategy")
ax.set_ylabel("Grid Draw [kW]")
ax.set_xticklabels(models)
#ax.set(ylim=(0, 24))

if writefiles:
    plt.savefig("output/peakshaving_strategies.png",
                bbox_inches='tight', dpi=300)
    #plt.close()


## Running time

In [ ]:
data = {}

#read logfile line by line
with open("../../data/lila/computations.log", 'r') as logfile:
    for line in logfile:
        split1 = line.split(' - ')
        timestamp = pd.Timestamp(split1[0])
        calctime = float(split1[3].split(': ')[2].split(' ')[0])
        data[timestamp] = calctime

#convert dictionary to dataframe
logdata = pd.DataFrame.from_dict(data, orient="index", columns=["calctime"])
logdata.index.name = "runningdate"

#remove index
logdata.reset_index(inplace=True)

#convert string to datetime
logdata["runningdate"] =  pd.to_datetime(logdata["runningdate"])

#workaround to missing floor function in pandas
logdata["runningdate"] = logdata.runningdate - pd.Timedelta(freq) # workaround to missing floor function in pandas
logdata["runningdate"] = logdata["runningdate"].round(freq)
logdata = logdata[(logdata["runningdate"] >= phases["phase1"]) & (logdata["runningdate"] < phases["end"])]

#restore index
logdata.set_index("runningdate", inplace=True)

#add field test phases
logdata["phase"] = 0
logdata.loc[phases["phase1"]:phases["phase2"], "phase"] = 1
logdata.loc[phases["phase2"]:phases["phase3"], "phase"] = 2
logdata.loc[phases["phase3"]:phases["phase4"], "phase"] = 3
logdata.loc[phases["phase4"]:phases["end"], "phase"] = 4

logdata.head()



In [ ]:
dfrt_pivot_strategies = logdata.pivot_table(values="calctime", index="phase", aggfunc=["max", "min", "std", "mean"]).round(2)
models = ["Direct", "Rule-based", "Predictive", "Stochastic"]
dfrt_pivot_strategies.index = models
dfrt_pivot_strategies.columns.set_levels(["Maximum", "Minimum", "Std. Deviation", "Average"], level=0, inplace=True)
dfrt_pivot_strategies.columns.set_levels(["Computation Time [s]"], level=1, inplace=True)
dfrt_pivot_strategies

In [ ]:
if writefiles:
    dfrt_pivot_strategies.to_latex(
        buf="output/computationtime_strategies.tex", bold_rows=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")

f, ax = plt.subplots()
sns.violinplot(data=logdata, x="phase", y="calctime",
               alpha=1, bw=.2, cut=1, linewidth=2)
ax.set_xlabel("Charging Strategy")
ax.set_ylabel("Computation Time [s]")
ax.set_yscale("log")
ax.set_xticklabels(models)
#ax.set(ylim=(0, 1000))

if writefiles:
    plt.savefig("output/computationtime_strategies.png",
                bbox_inches='tight', dpi=300)
    #plt.close()
